# Cálculo del Click-Through-Rate (CTR)

Este dataset está tomado de <a href="https://www.kaggle.com/c/kddcup2012-track2/overview">Kaggle</a>

La publicidad en búsquedas ha sido una de las principales fuentes de ingresos de la industria de Internet durante años. Una tecnología clave detrás de la publicidad en búsquedas es predecir la tasa de clics (pCTR) de los anuncios, ya que el modelo económico detrás de la publicidad requiere valores de pCTR para clasificar los anuncios y fijar los precios. En esta tarea, dados ejemplos de entrenamiento obtenidos de logs de sesión del motor de búsqueda propietario de Tencent, soso.com, los participantes deben predecir con precisión el pCTR de los anuncios en los ejemplos de test.

**DATOS DE ENTRENAMIENTO**

El archivo de datos de entrenamiento es un archivo de texto, donde cada línea es una instancia de entrenamiento de los logs de sesiones de búsqueda. Para comprender los logs, describimos las sesiones de búsqueda.

Una sesión de búsqueda se refiere a una interacción entre un usuario y el motor de búsqueda. Contiene los siguientes ingredientes: el usuario, la consulta emitida por el usuario, algunos anuncios devueltos por el motor de búsqueda y, por lo tanto, impresionados (mostrado) al usuario, y cero o más anuncios en los que el usuario hizo clic. Para mayor claridad, presentamos una terminología aquí. El número de anuncios mostrados en una sesión se conoce como "profundidad". El orden de un anuncio en la lista de impresiones se conoce como la "posición" de ese anuncio. Un anuncio, cuando esté mostrado, está compuesto de un texto corto conocido como "título", seguido de un texto un poco más largo conocido como "descripción" y una URL (generalmente abreviada para ahorrar espacio en la pantalla) conocida como "URL visible".

Dividimos cada sesión en varias instancias, donde cada instancia describe un anuncio impreso en una determinada configuración (es decir, con ciertos valores de profundidad y posición). Agregamos instancias con el mismo ID de usuario, ID de anuncio, consulta y configuración para reducir el tamaño del conjunto de datos. Por lo tanto, esquemáticamente, cada fila contiene al menos la siguiente información:

UserID 
AdID 
QueryID
Depth 
Position 
Impression: número de sesiones en los que el anuncio AdID fue mostrado al usuario UserID que realizó la consulta QueryID
Click : número de veces, de entre las impresiones, que el usuario UserID hizo click en el anuncio AdID

Además, los datos de training, validación y test contienen más información que la lista anterior, porque cada anuncio y cada usuario tienen algunas propiedades adicionales. Incluimos algunas de estas propiedades en los ejemplos de training, validación y test, y colocamos otras propiedades en archivos de datos separados que se pueden indexar utilizando los campos de identificadores de los ejemplos. Para obtener más información sobre estos archivos de datos, consulte la sección ARCHIVOS DE DATOS ADICIONALES.

Finalmente, después de incluir características adicionales, cada instancia de entrenamiento es una línea que consta de campos delimitados por un carácter de espacio en blanco:

1. Click: como se describe en la lista anterior.
2. Impression: como se describe en la lista anterior.
3. DisplayURL: una propiedad del anuncio. La URL se muestra junto con el título y la descripción de un anuncio. Suele ser la URL acortada de la página de destino del anuncio, pero no siempre. En el archivo de datos, esta URL se codifica para mantener el anonimato.
4. AdID: como se describe en la lista anterior.
5. AdvertiserID: una propiedad del anuncio. Algunos anunciantes optimizan constantemente sus anuncios, por lo que el título y la descripción de sus anuncios son más atractivos que los de los anuncios de otros.
6. Depth: (profundidad) una propiedad de la sesión, como se describió anteriormente.
7. Posición: una propiedad de un anuncio en una sesión, como se describió anteriormente.
8. QueryID: id de la consulta. Esta identificación es un valor que empieza en 0. Es la clave del archivo de datos 'queryid_tokensid.txt'.
9. KeywordID: una propiedad de los anuncios. Esta es la clave de 'buyerkeyword_tokensid.txt'.
10. TitleID: una propiedad de los anuncios. Esta es la clave de 'titleid_tokensid.txt'.
11. DescripciónID: una propiedad de los anuncios. Esta es la clave de 'descriptionid_tokensid.txt'.
12. ID de usuario. Esta es la clave de 'userid_profile.txt'. Cuando no podemos identificar al usuario, este campo tiene un valor especial de 0.

**FICHEROS ADICIONALES**

De los ficheros adicionales solo utilizaremos:

* userid_profile.txt: Each line of ‘userid_profile.txt’ is composed of UserID, Gender, and Age, delimited by the TAB character. Note that not every UserID in the training and the testing set will be present in ‘userid_profile.txt’. Each field is described below: 

  1. Gender (género): '1' para masculino, '2' para femenino,  y '0'  para desconocido. 
  2. Age (edad): '1'  para el intervalo (0, 12],  '2' para (12, 18], '3' para (18, 24], '4'  para  (24, 30], '5' para (30,  40], y '6' para mayor de 40.
  
**AJUSTE DE UN MODELO DE PREDICCIÓN**

Para más información puede consultarse el artículo escrito por el equipo ganador de la competición [aquí](https://jyunyu.csie.org/docs/pubs/kddcup2012paper.pdf). Como puede comprobarse, utilizan variables creadas a partir de un pequeño procesamiento de lenguaje natural aplicado al título y la descripción de los anuncios. El proceso de ingeniería de variables que llevan a cabo es fundamental para sus resultados.


In [6]:
clicksDF = spark.read.option("inferSchema", "true")\
                     .option("sep", "\t")\
                     .csv("gs://ucmbucket/data/ctr_training.txt")\
                     .toDF("Click", "Impression", "DisplayURL", "AdID", "AdvertiserID", "Depth",
                        "Position", "QueryID", "KeywordID", "TitleID", "DescriptionID", "UserID")\
                     .cache()
clicksDF.show()

+-----+----------+--------------------+--------+------------+-----+--------+--------+---------+-------+-------------+------+
|Click|Impression|          DisplayURL|    AdID|AdvertiserID|Depth|Position| QueryID|KeywordID|TitleID|DescriptionID|UserID|
+-----+----------+--------------------+--------+------------+-----+--------+--------+---------+-------+-------------+------+
|    0|         1| 4298118681424644510| 7686695|         385|    3|       3|    1601|     5521|   7709|          576|490234|
|    0|         1| 4860571499428580850|21560664|       37484|    2|       2| 2255103|      317|  48989|        44771|490234|
|    0|         1| 9704320783495875564|21748480|       36759|    3|       3| 4532751|    60721| 685038|        29681|490234|
|    0|         1|13677630321509009335| 3517124|       23778|    3|       1|    1601|     2155|   1207|         1422|490234|
|    0|         1| 3284760244799604489|20758093|       34535|    1|       1| 4532751|    77819| 266618|       222223|490234|


<div class="alert alert-block alert-success">
<b>TU TURNO</b>:
    <ul>
        <li>¿Cuántos usuarios distintos y anuncios distintos hay? ¿Cuántos anunciantes?             
        <li> ¿Cuál es el número medio de anuncios distintos visualizados por cada usuario?
        <li> ¿Cuál es el número medio de anunciantes vistos por un usuario?
        <li> Calcular el CTR (Click-through rate) para cada usuario como el número total de clicks del usuario entre el número total de impresiones para ese usuario.
        <li> En esa operación, incluir también el recuento del número de anuncios vistos por el usuario. ¿Tiene relación con el CTR?
        <li> Utilizando el dataset `userid_profile.txt` de la edad y el grupo de los usuarios, obtener el CTR medio de cada grupo de usuarios a partir de una agregación en el grupo de todos los CTR individuales de los usuarios de ese grupo.
        <li> Repetir la operación utilizando exclusivamente una consulta en SQL.
    </ul>
</div>

<div class="alert alert-block alert-success">
<b>AJUSTE DE UN MODELO PREDICTIVO</b>:
Vamos a ajustar un modelo de predicción como un algoritmo de clasificación binaria que nos da la probabilidad de que el usuario haga click en el anuncio. Para ello:

* Escribir una UDF para crear una nueva columna `targetV` de tipo vector que, para cada fila, contenga un **vector de números reales** de dimensión `Impressions` que sean 0.0 o 1.0. En dicho vector, los `Clicks` primeros elementos deben ser 1.0 y los restantes `Impressions-Clicks` elementos deben ser 0.0. El tipo devuelto por la UDF debe ser `ArrayType(DoubleType())` por lo que habrá que importar `ArrayType` y `DoubleType` de `pyspark.sql.types`.
* Utilizar `withColumn` combinado con la función `F.explode(F.col("targetV"))` para repetir cada fila tantas veces como elementos tenga su vector. La nueva columna generada a partir del vector pasará a contener solamente un 0.0 o un 1.0. Esta utilización de `withColumn` es uno de los pocos casos en los que el DF resultante **no** tiene el mismo número de filas que el original.
* Ajustar un algoritmo de <b>Regresión Logística</b> a esta variable target. Incluir los pasos de transformación necesarios para las variables categóricas (Gender, AdID, AdvertiserID) y encapsularlo todo en un `Pipeline`. Entrenar con una muestra aleatoria de, por ejemplo, el 10 % de los ejemplos si comprbamos que el entrenamiento tardase demasiado al probar con todos los datos, debido a que nuestro cluster es pequeño.
* Ajustar los hiper-parámetros de la regresión logística (en especial, <b>regParam</b>) con Spark.
</div>

